In [ ]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [1]:
import os
for dirname, _, filenames in os.walk(r'C:/Users/Admin/drug review/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Admin/drug review/dataset\drugsComTest_raw.csv
C:/Users/Admin/drug review/dataset\drugsComTrain_raw.csv


In [3]:
#reading the data

test_csv = pd.read_csv(r'C:/Users/Admin/drug review\drugsComTest_raw.csv')
train_csv = pd.read_csv(r'C:/Users/Admin/drug review\drugsComTrain_raw.csv')

# EDA

In [4]:
#stopword removal and lemmatization

stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [5]:
# Preprocessing
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
lemmatizer

<WordNetLemmatizer>

In [7]:
train_csv.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [8]:
# 1 - positive sentiment if ratings >= 5
# 0 - negative sentiment if ratings < 5
train_csv['label'] = -1
for i in range(0, len(train_csv)):
    if(train_csv['rating'][i]) >= 5:
        train_csv['label'][i] = 1
    else:
        train_csv['label'][i] = 0

C:\Users\Admin\AppData\Local\Temp/ipykernel_1848/2139510790.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_csv['label'][i] = 1
C:\Users\Admin\AppData\Local\Temp/ipykernel_1848/2139510790.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_csv['label'][i] = 0


In [9]:
train_csv.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,label
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,1
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,1
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,1
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,1
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,1


In [10]:
# 1 - positive sentiment if ratings >= 5
# 0 - negative sentiment if ratings < 5
test_csv['label'] = -1
for j in range(0, len(test_csv)):
    if(test_csv['rating'][j]) >= 5:
        test_csv['label'][j] = 1
    else:
        test_csv['label'][j] = 0
        
test_csv.tail()

C:\Users\Admin\AppData\Local\Temp/ipykernel_1848/1653628789.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_csv['label'][j] = 1
C:\Users\Admin\AppData\Local\Temp/ipykernel_1848/1653628789.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_csv['label'][j] = 0


,uniqueID,drugName,condition,review,rating,date,usefulCount,label
53761,159999,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10,13-Sep-14,43,1
53762,140714,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9,8-Oct-16,11,1
53763,130945,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8,15-Nov-10,7,1
53764,47656,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1,28-Nov-11,20,0
53765,113712,Arthrotec,Sciatica,"""It works!!!""",9,13-Sep-09,46,1


In [11]:
a = -1
set(test_csv['label'])

{0, 1}

In [13]:
train_X_non = train_csv['review']   # '0' refers to the review text
train_y = train_csv['label']   # '1' corresponds to Label (1 - positive and 0 - negative)
test_X_non = test_csv['review']
test_y = test_csv['label']

train_X=[]
test_X=[]

# text preprocessing

In [14]:
#text pre processing

for i in range(0, len(train_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', train_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

In [48]:
#text pre processing

for i in range(0, len(test_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', test_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [70]:
len(train_X)

322594

In [50]:
len(test_X)

53766

In [71]:
#tf idf - converting texts to numeric vector

tf_idf = TfidfVectorizer()
tf_idf

TfidfVectorizer()

In [52]:
#applying tf idf to training data

X_train_tf = tf_idf.fit_transform(train_X)
X_train_tf

<161297x41933 sparse matrix of type '<class 'numpy.float64'>'
	with 5637702 stored elements in Compressed Sparse Row format>

In [53]:
#applying tf idf to training data

X_train_tf = tf_idf.transform(train_X)
X_train_tf

<161297x41933 sparse matrix of type '<class 'numpy.float64'>'
	with 5637702 stored elements in Compressed Sparse Row format>

In [54]:
# check the dimensions of the train data

print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 161297, n_features: 41933


In [55]:
#transforming test data into tf-idf matrix

X_test_tf = tf_idf.transform(test_X)
X_test_tf

<53766x41933 sparse matrix of type '<class 'numpy.float64'>'
	with 1872680 stored elements in Compressed Sparse Row format>

In [56]:
# check the dimensions of the train data

print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 53766, n_features: 41933


# Model Creation - NB classifier

In [57]:
train_y=train_y.astype('int')

In [58]:
train_y

0         1
1         1
2         1
3         1
4         1
         ..
161292    1
161293    0
161294    0
161295    1
161296    1
Name: label, Length: 161297, dtype: int32

In [59]:
#naive bayes classifier

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

MultinomialNB()

In [60]:
#predicted y

y_pred = naive_bayes_classifier.predict(X_test_tf)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [61]:
print(metrics.classification_report(test_y, y_pred))
#                                     target_names=['Positive', 'Negative']))

              precision    recall  f1-score   support

           0       0.91      0.12      0.22     13497
           1       0.77      1.00      0.87     40269

    accuracy                           0.78     53766
   macro avg       0.84      0.56      0.54     53766
weighted avg       0.81      0.78      0.71     53766



In [62]:
print("Confusion matrix:")
print(metrics.confusion_matrix(test_y, y_pred))

Confusion matrix:
[[ 1647 11850]
 [  166 40103]]


In [63]:
test = ["This is unlike any kind of adventure movie my eyes have ever seen in such a long time, the characters, the musical score for every scene, the story, the beauty of the landscapes of Pandora, the rich variety and uniqueness of the flora and fauna of Pandora, the ways and cultures and language of the natives of Pandora, everything about this movie I am beyond impressed and truly captivated by. Sam Worthington is by far my favorite actor in this movie along with his character Jake Sulley, just as he was a very inspiring actor in The Shack Sam Worthington once again makes an unbelievable mark in one of the greatest and most captivating movies you'll ever see. "]
review = re.sub('[^a-zA-Z]', ' ', test[0])
review = review.lower()
review = review.split()
review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
test_processed =[ ' '.join(review)]


In [64]:
test_processed = ['unlike kind adventure movie eye ever seen long time character musical score every scene story beauty landscape pandora rich variety uniqueness flora fauna pandora way culture language native pandora everything movie beyond impressed truly captivated sam worthington far favorite actor movie along character jake sulley inspiring actor shack sam worthington make unbelievable mark one greatest captivating movie ever see']

In [65]:
test_input = tf_idf.transform(test_processed)

In [66]:
test_input.shape

(1, 41933)

In [67]:
res=naive_bayes_classifier.predict(test_input)[0]

if res==1:
    print("Good Review")
    
elif res==0:
    print("Bad Review")

Good Review


SyntaxError: invalid syntax. Perhaps you forgot a comma? (Temp/ipykernel_6032/2906897303.py, line 1)